<a href="https://colab.research.google.com/github/scsanjay/ml_from_scratch/blob/main/03.%20Naive%20Bayes/MultinomialNaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of Multinomial Naive Bayes

In [ ]:
import numpy as np

In [ ]:
class MultinomialNaiveBayes:
  """
  Parameters
  ----------
  alpha : float, default=1.0

  fit_prior : bool, default=True

  class_prior : array-like of shape (n_classes,), default=None

  Attributes
  ----------
  class_count_ : ndarray of shape (n_classes,)

  class_log_prior_ : ndarray of shape (n_classes, )

  classes_ : ndarray of shape (n_classes,)
  
  n_classes_ : int

  feature_count_ : ndarray of shape (n_classes, n_features)

  feature_log_prob_ : ndarray of shape (n_classes, n_features)

  n_features_ : int
  """

  def __init__(self, alpha=1.0, fit_prior=True, class_prior=None):
    self.alpha = alpha
    self.fit_prior = fit_prior
    self.class_prior = class_prior

  def fit(self, X, y):
    """
    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)
    y : array-like of shape (n_samples,)

    Returns
    -------
    self : object
    """
    # convert train data to numpy array if in other form
    X_train = np.array(X)
    y_train = np.array(y)
    
    n_samples = len(y_train)

    # get distinct class labels
    self.classes_ = np.sort(np.unique(y_train))

    # get total number of class available
    self.n_classes_ = len(self.classes_)

    # get frequency for each class
    self.class_count_ = np.zeros(self.n_classes_)
    for idx, class_ in enumerate(self.classes_):
      self.class_count_[idx] = np.count_nonzero(y_train == class_)
    
    # get log priors
    self.class_log_prior_ = np.zeros(self.n_classes_)
    if self.class_prior is not None:
      self.class_log_prior_ = np.log(np.array(self.class_prior))
    elif self.fit_prior == False:
      self.class_log_prior_ = np.full(self.n_classes, -np.log(self.n_classes))
    else:
      self.class_log_prior_ = np.log(self.class_count_/n_samples)

    # number of features
    self.n_features_ = X_train.shape[1]

    # get feature counts and log likelihood probabilities
    # for each class and each features
    self.feature_count_ = np.zeros((self.n_classes_, self.n_features_))
    self.feature_log_prob_ = np.zeros((self.n_classes_, self.n_features_))
    for i, class_ in enumerate(self.classes_):
      # get data according to class
      temp_data = X_train[np.where(y_train==class_)]
      self.feature_count_[i] = np.sum(temp_data, axis=0)
      self.feature_log_prob_[i] = np.log(
                                (self.feature_count_[i]+self.alpha)/
                                (np.sum(self.feature_count_[i])+self.alpha*self.n_features_)
                                )
    return self

  def predict(self, X):
    """
    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)

    Returns
    -------
    C : ndarray of shape (n_samples,)
    """
    # convert test data to numpy array if in other form
    X_test = np.array(X)

    y_pred = np.empty(len(X_test))

    #predict class for each test data
    for idx, x in enumerate(X_test):
      # SUM(xij*log_likelyhood(fij))+log_prior(j) 
      # where j is class and fij is probability of feature i given j class
      y_pred[idx] = self.classes_[np.argmax(np.dot(x, self.feature_log_prob_.T) + self.class_log_prior_)]
    
    return y_pred
      


# Compare the implementation with sklearn.naive_bayes.MultinomialNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# Let's create some data
X_train = np.array([
  [2,1,3,1,0],
  [1,3,2,0,1],
  [0,0,1,2,3],
  [1,0,0,3,1],
  [1,0,0,2,2]
 ])
y_train = np.array([1, 1, 0, 0, 0])

X_test = np.array([
  [3,1,2,1,0],
  [0,1,0,1,3]
 ])

my_clf is object of the implemented Multinomial Naive Bayes

In [ ]:
my_clf = MultinomialNaiveBayes()
my_clf.fit(X_train, y_train)

In [ ]:
help(my_clf)

Help on MultinomialNaiveBayes in module __main__ object:

class MultinomialNaiveBayes(builtins.object)
 |  MultinomialNaiveBayes(alpha=1.0, fit_prior=True, class_prior=None)
 |  
 |  Parameters
 |  ----------
 |  alpha : float, default=1.0
 |  
 |  fit_prior : bool, default=True
 |  
 |  class_prior : array-like of shape (n_classes,), default=None
 |  
 |  Attributes
 |  ----------
 |  class_count_ : ndarray of shape (n_classes,)
 |  
 |  class_log_prior_ : ndarray of shape (n_classes, )
 |  
 |  classes_ : ndarray of shape (n_classes,)
 |  
 |  n_classes_ : int
 |  
 |  feature_count_ : ndarray of shape (n_classes, n_features)
 |  
 |  feature_log_prob_ : ndarray of shape (n_classes, n_features)
 |  
 |  n_features_ : int
 |  
 |  Methods defined here:
 |  
 |  __init__(self, alpha=1.0, fit_prior=True, class_prior=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  fit(self, X, y)
 |      Parameters
 |      ----------
 |      X : array-like of shape (

clf is object of sklearn's implementation

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Let's compare the attributes

In [ ]:
print(my_clf.class_log_prior_)
print(clf.class_log_prior_)

[-0.51082562 -0.91629073]
[-0.51082562 -0.91629073]


In [ ]:
print(my_clf.feature_count_)
print(clf.feature_count_)

[[2. 0. 1. 7. 6.]
 [3. 4. 5. 1. 1.]]
[[2. 0. 1. 7. 6.]
 [3. 4. 5. 1. 1.]]


In [ ]:
print(my_clf.feature_log_prob_)
print(clf.feature_log_prob_)

[[-1.94591015 -3.04452244 -2.35137526 -0.9650809  -1.09861229]
 [-1.55814462 -1.33500107 -1.15267951 -2.2512918  -2.2512918 ]]
[[-1.94591015 -3.04452244 -2.35137526 -0.9650809  -1.09861229]
 [-1.55814462 -1.33500107 -1.15267951 -2.2512918  -2.2512918 ]]


### Let's compare the predict

In [ ]:
my_clf.predict(X_test)

array([1., 0.])

In [ ]:
clf.predict(X_test)

array([1, 0])

### Let's try setting prior

In [ ]:
my_clf = MultinomialNaiveBayes(class_prior=[1000,1])
my_clf.fit(X_train, y_train)
my_clf.predict(X_test)

array([0., 0.])

In [ ]:
clf = MultinomialNB(class_prior=[1000,1])
clf.fit(X_train, y_train)
clf.predict(X_test)

array([0, 0])

### Let's try with aplha=10

In [ ]:
my_clf = MultinomialNaiveBayes(alpha=10)
my_clf.fit(X_train, y_train)
print(my_clf.feature_log_prob_)

[[-1.70474809 -1.88706965 -1.79175947 -1.3564414  -1.41706602]
 [-1.59393373 -1.51982575 -1.45083288 -1.76098781 -1.76098781]]


In [ ]:
clf = MultinomialNB(alpha=10)
clf.fit(X_train, y_train)
print(clf.feature_log_prob_)

[[-1.70474809 -1.88706965 -1.79175947 -1.3564414  -1.41706602]
 [-1.59393373 -1.51982575 -1.45083288 -1.76098781 -1.76098781]]


## Everything seems to be working fine